# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -U scikit-learn

import sklearn
sklearn.__version__

    100% |████████████████████████████████| 7.1MB 3.3MB/s eta 0:00:01    30% |█████████▉                      | 2.2MB 19.8MB/s eta 0:00:01    45% |██████████████▌                 | 3.2MB 24.6MB/s eta 0:00:01    84% |███████████████████████████     | 5.9MB 31.3MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.19.1
    Uninstalling scikit-learn-0.19.1:
      Successfully uninstalled scikit-learn-0.19.1


'0.22.1'

In [3]:
# import libraries
from pathlib import Path
from sqlalchemy import create_engine
import re
import pickle

import nltk
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.linear_model import LogisticRegression

import numpy as np
import pandas as pd

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterData', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
cols = ['message', 'genre']
X = df[cols]
Y = df.loc[:, 'related':'direct_report']
X.shape, Y.shape

((26216, 2), (26216, 36))

In [7]:
Y = Y.drop('child_alone', axis=1)
Y.shape

(26216, 35)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
#     clean_text = re.sub('\W', ' ', text).lower()
    tokens = word_tokenize(clean_text)
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
RANDOM_STATE = 42

In [10]:
text_col = 'message'
text_pipe = Pipeline([
    ('tfidf', TfidfVectorizer())
])

cat_cols = ['genre']
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder())
])

preproc_pipe = ColumnTransformer([
    ('text', text_pipe, text_col),
#     ('cat', cat_pipe, cat_cols)
], n_jobs=-1)

pipeline = Pipeline([
    ('preprocess', preproc_pipe),
    ('clf', MultiOutputClassifier(LogisticRegression(solver='liblinear', random_state=RANDOM_STATE)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((20972, 2), (5244, 2), (20972, 35), (5244, 35))

In [12]:
Y_train.apply(pd.value_counts, normalize=True)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.238604,0.829344,0.995613,0.583778,0.920751,0.950267,0.971533,0.981738,0.966384,0.936439,...,0.988461,0.955751,0.720628,0.916842,0.906685,0.989081,0.905731,0.980307,0.947549,0.806695
1,0.761396,0.170656,0.004387,0.416222,0.079249,0.049733,0.028467,0.018262,0.033616,0.063561,...,0.011539,0.044249,0.279372,0.083158,0.093315,0.010919,0.094269,0.019693,0.052451,0.193305


In [13]:
Y_test.apply(pd.value_counts, normalize=True)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.249047,0.829329,0.995042,0.593631,0.919527,0.948513,0.975782,0.983219,0.970442,0.935355,...,0.987223,0.957475,0.725782,0.921625,0.907323,0.989893,0.908848,0.977689,0.947368,0.805301
1,0.750953,0.170671,0.004958,0.406369,0.080473,0.051487,0.024218,0.016781,0.029558,0.064645,...,0.012777,0.042525,0.274218,0.078375,0.092677,0.010107,0.091152,0.022311,0.052632,0.194699


In [14]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=-1, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  Pipeline(memory=None,
                                                           steps=[('tfidf',
                                                                   TfidfVectorizer(analyzer='word',
                                                                                   binary=False,
                                                                                   decode_error='strict',
                                                                                   dtype=<class 'numpy.float64'>,
                                                                                   encoding='utf-8',
                                         

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
Y_pred_test = pipeline.predict(X_test)

average_modes = ['micro', 'macro', 'samples', 'weighted']

for average in average_modes:
    print(average)
    print('Accuracy: ', accuracy_score(Y_test, Y_pred_test))
    print('F1 score: ', f1_score(Y_test, Y_pred_test, average=average))
    print('Precision score: ', precision_score(Y_test, Y_pred_test, average=average))
    print('Recall score: ', recall_score(Y_test, Y_pred_test, average=average))
    print()

micro
Accuracy:  0.288520213577
F1 score:  0.647153874403
Precision score:  0.815304894975
Recall score:  0.536503683858

macro
Accuracy:  0.288520213577
F1 score:  0.280032337876
Precision score:  0.572352617362
Recall score:  0.21975312065

samples
Accuracy:  0.288520213577


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetr

F1 score:  0.505928336649
Precision score:  0.658229056942
Recall score:  0.475727035552

weighted
Accuracy:  0.288520213577
F1 score:  0.584911126295
Precision score:  0.760188633254
Recall score:  0.536503683858



In [19]:
scores_list = []

for i, col_name in enumerate(Y_test.columns):
    print(col_name)
    
    score = accuracy_score(Y_test.iloc[:, i], Y_pred_test[:, i])
    scores_list.append(score)
    print(score)
    
    print(classification_report(Y_test.iloc[:, i], Y_pred_test[:, i]))

related
0.818077803204
              precision    recall  f1-score   support

           0       0.75      0.41      0.53      1306
           1       0.83      0.95      0.89      3938

    accuracy                           0.82      5244
   macro avg       0.79      0.68      0.71      5244
weighted avg       0.81      0.82      0.80      5244

request
0.90465293669
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4349
           1       0.83      0.55      0.66       895

    accuracy                           0.90      5244
   macro avg       0.87      0.77      0.80      5244
weighted avg       0.90      0.90      0.90      5244

offer
0.995041952708
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5004
           1       0.83      0.08      0.14       240

    accuracy                           0.96      5244
   macro avg       0.89      0.54      0.56      5244
weighted avg       0.95      0.96      0.94      5244

buildings
0.955568268497
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4977
           1       0.81      0.16      0.27       267

    accuracy                           0.96      5244
   macro avg       0.89      0.58      0.63      5244
weighted avg       0.95      0.96      0.94      5244

electricity
0.978070175439
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5122
           1       0.89      0.07      0.12       122

    accuracy                           0.98      5244
   macro avg       0.93      0.53      0.56      5244
weighted avg       0.9

In [20]:
training_classes_order = list(reversed(np.argsort(scores_list)))
print(training_classes_order)

[24, 2, 22, 23, 30, 14, 12, 25, 7, 13, 32, 21, 6, 8, 15, 16, 31, 9, 26, 19, 20, 5, 28, 33, 10, 11, 18, 29, 4, 1, 17, 27, 34, 0, 3]


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
# TODO Create unified param_grid for 3 cases

# 0.19564213188603344
param_grid_dummy = {
    'clf': [MultiOutputClassifier(DummyClassifier(), n_jobs=-1)],
    'clf__estimator__random_state': [RANDOM_STATE],
    'clf__estimator__strategy': ['most_frequent']
}

# 0.26311308899302172
param_grid_multi = {
    'clf': [MultiOutputClassifier(LogisticRegression(), n_jobs=-1)],
    'clf__estimator__solver': ['liblinear'],
    'clf__estimator__random_state': [RANDOM_STATE],
    'clf__estimator__C': [1],
    'clf__estimator__max_iter': [100]
}

# random: 0.31441978640430607
# training_classes_order: 0.32648326307132264
# training_classes_order[::-1]: 0.31537320924982437

param_grid_chain = {
    'clf': [ClassifierChain(LogisticRegression())],
    'clf__order': [training_classes_order],
    'clf__random_state': [RANDOM_STATE],
    'clf__base_estimator__solver': ['liblinear'],
    'clf__base_estimator__random_state': [RANDOM_STATE],
    'clf__base_estimator__C': [1],
    'clf__base_estimator__max_iter': [100]
}

cv = KFold(n_splits=3)
grid_search = GridSearchCV(pipeline, param_grid_chain, scoring='accuracy', 
                           cv=cv, verbose=0, n_jobs=-1)
grid_search.fit(X_train, Y_train)
grid_search.best_score_, grid_search.best_params_

(0.32648326307132264,
 {'clf': ClassifierChain(base_estimator=LogisticRegression(C=1, class_weight=None,
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='auto',
                                                    n_jobs=None, penalty='l2',
                                                    random_state=42,
                                                    solver='liblinear',
                                                    tol=0.0001, verbose=0,
                                                    warm_start=False),
                  cv=None,
                  order=[24, 2, 22, 23, 30, 14, 12, 25, 7, 13, 32, 21, 6, 8, 15,
                         16, 31, 9, 26, 19, 20, 5, 28, 33, 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
Y_pred_test = grid_search.predict(X_test)

average_modes = ['micro', 'macro', 'samples', 'weighted']

for average in average_modes:
    print(average)
    print('Accuracy: ', accuracy_score(Y_test, Y_pred_test))
    print('F1 score: ', f1_score(Y_test, Y_pred_test, average=average, zero_division=0))
    print('Precision score: ', precision_score(Y_test, Y_pred_test, average=average, zero_division=0))
    print('Recall score: ', recall_score(Y_test, Y_pred_test, average=average, zero_division=0))
    print()

micro
Accuracy:  0.343440122044
F1 score:  0.53987524595
Precision score:  0.863879957128
Recall score:  0.392620105949

macro
Accuracy:  0.343440122044
F1 score:  0.225666666856
Precision score:  0.580371496122
Recall score:  0.162468213826

samples
Accuracy:  0.343440122044
F1 score:  0.383224941585
Precision score:  0.577696959791
Recall score:  0.344578022799

weighted
Accuracy:  0.343440122044
F1 score:  0.483473201823
Precision score:  0.790476644382
Recall score:  0.392620105949



In [23]:
for i, col_name in enumerate(Y_test.columns):
    print(col_name)
    print(classification_report(Y_test.iloc[:, i], Y_pred_test[:, i]))

related
              precision    recall  f1-score   support

           0       0.55      0.71      0.62      1306
           1       0.89      0.81      0.85      3938

    accuracy                           0.78      5244
   macro avg       0.72      0.76      0.73      5244
weighted avg       0.81      0.78      0.79      5244

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4349
           1       0.87      0.47      0.61       895

    accuracy                           0.90      5244
   macro avg       0.88      0.73      0.78      5244
weighted avg       0.90      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      524

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4916
           1       0.00      0.00      0.00       328

    accuracy                           0.94      5244
   macro avg       0.47      0.50      0.48      5244
weighted avg       0.88      0.94      0.91      5244

transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5004
           1       0.83      0.04      0.08       240

    accuracy                           0.96      5244
   macro avg       0.89      0.52      0.53      5244
weighted avg       0.95      0.96      0.94      5244

buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4977
           1       0.92      0.09      0.16       267

    accuracy                           0.95      5244
   macro avg       0.94      0.54      0.57      5244
weighted avg       0.95      0.95      0.93      5244


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [32]:
PICKLE_PATH = Path('./pickle')
PICKLE_PATH.mkdir(parents=True, exist_ok=True)

In [34]:
with open(PICKLE_PATH/'model.pkl', 'wb') as f:
    pickle.dump(grid_search.best_estimator_, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.